In [1]:
import os
from openai import OpenAI
import dotenv
dotenv.load_dotenv()

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

In [2]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "",
        }
    ],
    model="gpt-4o",
)

str(chat_completion.choices[0].message.content)


'Hello! How can I assist you today?'

In [3]:

# go through sample.json and ask questions to gpt
import json
all_qs = json.load(open("filtered_train.json"))

# randomly sample 10 questions
import random
sample = random.sample(all_qs, 20)

print(sample)

output = []

for i in sample:
    # print("-"*20)
    ootb_llm_rseponse = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model="gpt-4o"
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Rewrite this question replacing all questions with a what, but retain the meaning by specifying what entity or what person or what timeframe the \"what\" answering: {i['nq_question']}",
            }
        ],
        model="gpt-4o"
    )

    print(chat_completion.choices[0].message.content)


    final_answer = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {chat_completion.choices[0].message.content}",
            }
        ],
        model="gpt-4o"
    )

    curr = {
        "data_id": i["nq_id"],
        "ambig_question": i["nq_question"],
        "ambig_prompt_response": ootb_llm_rseponse.choices[0].message.content,
        "disambig_question": chat_completion.choices[0].message.content,
        "disambig_prompt_response": final_answer.choices[0].message.content,
        "ground_truth": i["nq_answer"],
    }

    output.append(curr)

    # save output to json file
    json.dump(output, open("disambig_prompt.json", "w"))

    print(curr)


[{'nq_id': '-6879428134571341185', 'nq_question': 'When does hotel transylvania part 3 come out?', 'nq_answer': ['July 13 , 2018'], 'ambig_questions': [{'question': 'When does the film hotel transylvania part 3 come out at AIAFF, the french film festival?', 'answer': ['June 13, 2018']}, {'question': 'When does the film hotel transylvania part 3 come out in the United States?', 'answer': ['July 13, 2018']}, {'question': 'When does hotel transylvania part 3 video game come out?', 'answer': ['July 10, 2018']}, {'question': 'When does the film hotel transylvania part 3 come out in China?', 'answer': ['August 17, 2018']}, {'question': 'When does the film hotel transylvania part 3 come out in Italy and the Phillippines?', 'answer': ['late August 2018']}, {'question': 'When does the film hotel transylvania part 3 come out in Japan and Poland?', 'answer': ['October 2018']}]}, {'nq_id': '804600793856177571', 'nq_question': 'When was daylight saving first introduced to australia?', 'nq_answer': 